<a href="https://colab.research.google.com/github/nitsundon/100DaysofML/blob/main/Day01/LSTMrev02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import numpy as np


In [ ]:
df= pd.read_pickle("/content/drive/MyDrive/Libraries/pickle/preprocessed_demand_df.pkl")
df['datetime']=pd.to_datetime(df['datetime'])


In [ ]:
def create_datetime_features(df):


  df['block'] = 1+df['datetime'].dt.hour * 4 + df['datetime'].dt.minute // 15
  df['month'] = df['datetime'].dt.month
  df['day'] = df['datetime'].dt.day
  df['day_of_week'] = df['datetime'].dt.dayofweek
  df['year'] = df['datetime'].dt.year

  return df.copy();

In [ ]:
def add_cyclic_features(df, col, max_val,drop_original=True):
    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col]/max_val)
    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    if(drop_original):
      df.drop(col,axis=1,inplace=True)
    return df


In [ ]:
def inverse_cyclic_features(df, col, max_val):
    # Calculate the angle (theta) back from sin and cos
    theta = np.arctan2(df[f'{col}_sin'], df[f'{col}_cos'])

    # Normalize theta to be in [0, 2*pi]
    theta = (theta + 2 * np.pi) % (2 * np.pi)

    # Recover the original value
    df[col] = (theta * max_val) / (2 * np.pi)
    df[col]=df[col].astype(int)
    # Optionally, you can drop sin and cos columns if you want
    df.drop([f'{col}_sin', f'{col}_cos'], axis=1, inplace=True)

    return df

In [ ]:
def create_lags(df,lag_steps):
  for i in range(1,lag_steps+1):
    df[f'demand_lag_{i}']=df['demand'].shift(i)
  return df

In [ ]:
def create_lstm_feature_and_target(df):
  features = df.drop('demand', axis=1)
  target = df['demand']
  return features,target

In [ ]:
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,0])
    return np.array(dataX),np.array(dataY)




In [ ]:
df1=create_datetime_features(df)
df1.columns

In [ ]:
def inverse_scaled_features_df(df, scaler):
    df1 = df.copy()

    # Inverse cyclic features
    df1 = inverse_cyclic_features(df1, 'month', 12)
    df1 = inverse_cyclic_features(df1, 'day_of_week', 7)
    df1 = inverse_cyclic_features(df1, 'block', 96)
    df1 = inverse_cyclic_features(df1, 'day', 31)

    # Inverse scale selected columns
    cols_to_inverse = ['demand', 'year']
    s = scaler.inverse_transform(df1[cols_to_inverse])

    # Convert back to DataFrame to maintain structure
    s_df = pd.DataFrame(s, columns=cols_to_inverse, index=df1.index)

    # Update df1 with inverse transformed values
    df1[cols_to_inverse] = s_df

    return df1, scaler

In [ ]:
def create_scaled_features_df(df,scaler):
  df1=df.copy()
  df1=add_cyclic_features(df1, 'month', 12, drop_original=True)
  df1=add_cyclic_features(df1,'day_of_week', 7, drop_original=True)
  df1=add_cyclic_features(df1,'block', 96, drop_original=True)
  df1=add_cyclic_features(df1,'day',31, drop_original=True)

  s= scaler.fit_transform(df1[['demand','year']])
  df1['demand']=s[:,0]
  df1['year']=s[:,1]
  return df1,scaler

In [ ]:
def create_sequences_multi_output(data, seq_length, pred_length):
    X = []
    y = []
    for i in range(seq_length, len(data) - pred_length):
        X.append(data[i-seq_length:i, :-1])  # input features
        y.append(data[i:i+pred_length, -1])  # 192 future targets
    return np.array(X), np.array(y)



In [ ]:
scalar=MinMaxScaler()
df1,scalar=create_scaled_features_df(df1,scalar)
df1,type(df1)

In [ ]:
df1=inverse_scaled_features_df(df1,scalar)
type(df1)